In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
vacancy = pd.read_excel("../data/空置表.xlsx", parse_dates=["起始日期"])
vacancy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719 entries, 0 to 718
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   序号      719 non-null    int64         
 1   管理主体    719 non-null    object        
 2   项目名称    719 non-null    object        
 3   大厦名称    719 non-null    object        
 4   房号      719 non-null    object        
 5   用途      719 non-null    object        
 6   状态      708 non-null    object        
 7   面积      719 non-null    float64       
 8   使用率     708 non-null    float64       
 9   预计单价    719 non-null    float64       
 10  预计月租金   719 non-null    float64       
 11  起始日期    719 non-null    datetime64[ns]
 12  空置天数    719 non-null    int64         
 13  空置月数    718 non-null    float64       
 14  空置时长    719 non-null    object        
 15  责任人     526 non-null    object        
 16  空置原因    150 non-null    object        
 17  解决措施    94 non-null     object        
 18  计划解决时间  27

计算空置天数和控制月数. Pandas 专门提供了 Time deltas  来处理日期时间差. 两个日期时间相减会得到一个 dtype 是 `timedelta64[ns]`  对象来处理日期时间差的问题.

In [3]:
vacant_timedelta = pd.to_datetime("now") - vacancy["起始日期"]
vacant_timedelta

0     880 days 02:48:12.320248
1     149 days 02:48:12.320248
2     361 days 02:48:12.320248
3     149 days 02:48:12.320248
4      88 days 02:48:12.320248
                ...           
714   641 days 02:48:12.320248
715   641 days 02:48:12.320248
716   641 days 02:48:12.320248
717   641 days 02:48:12.320248
718    89 days 02:48:12.320248
Name: 起始日期, Length: 719, dtype: timedelta64[ns]

In [4]:
# 空置天数
vacant_days = (vacant_timedelta / np.timedelta64(1, "D")).astype("int")
vacant_days

0      880
1      149
2      361
3      149
4       88
      ... 
714    641
715    641
716    641
717    641
718     89
Name: 起始日期, Length: 719, dtype: int64

In [5]:
# 空置月数
vacant_month = round(vacant_timedelta / np.timedelta64(1, "M"), 2)
vacant_month

0      28.92
1       4.90
2      11.86
3       4.90
4       2.90
       ...  
714    21.06
715    21.06
716    21.06
717    21.06
718     2.93
Name: 起始日期, Length: 719, dtype: float64

In [6]:
vacancy["vacant_days"] = vacant_days
vacancy["vacant_month"] = vacant_month
vacancy.head()

,序号,管理主体,项目名称,大厦名称,房号,用途,状态,面积,使用率,预计单价,...,空置时长,责任人,空置原因,解决措施,计划解决时间,管理费,备注,红黄灯,vacant_days,vacant_month
0,1,园区运营中心,蛇口网谷,万融大厦,A座303/305,办公,简装,866.39,0.70,120.0,...,12个月以上,曹泽宇,NaN,NaN,NaN,NaN,NaN,NaN,880,28.92
1,2,园区运营中心,蛇口网谷,万融大厦,A401-402,办公,简装,1893.60,0.70,135.0,...,3-6个月,曹泽宇,NaN,NaN,NaN,NaN,已签约，9月1日生效,红灯,149,4.90
2,3,园区运营中心,蛇口网谷,万融大厦,BG01/G02/G06/G07/G08/G09/G10/G11/G12,商业,简装,986.06,0.75,240.0,...,6-12个月,商业小组,NaN,NaN,NaN,NaN,NaN,黄灯,361,11.86
3,4,园区运营中心,蛇口网谷,万融大厦,A401-402,办公,简装,1893.60,0.70,135.0,...,3-6个月,张可言,NaN,NaN,NaN,NaN,已签约，9月1日生效,NaN,149,4.90
4,5,园区运营中心,蛇口网谷,万融大厦,B座106A,办公,简装,211.90,0.75,135.0,...,3个月以内,曹泽宇,NaN,NaN,NaN,NaN,NaN,红灯,88,2.90


计算红黄灯

In [7]:
def lights(vacant_month):
    threshod_value_upper = 6
    threshod_value_down = 3
    
    if vacant_month >= threshod_value_upper:
        return 'R'
    elif vacant_month < threshod_value_down:
        return 'G'
    else:
        return 'Y'

light = vacancy.vacant_month.map(lights)
light

0      R
1      Y
2      R
3      Y
4      G
      ..
714    R
715    R
716    R
717    R
718    G
Name: vacant_month, Length: 719, dtype: object

In [8]:
vacancy["light"] = light
vacancy.head()

,序号,管理主体,项目名称,大厦名称,房号,用途,状态,面积,使用率,预计单价,...,责任人,空置原因,解决措施,计划解决时间,管理费,备注,红黄灯,vacant_days,vacant_month,light
0,1,园区运营中心,蛇口网谷,万融大厦,A座303/305,办公,简装,866.39,0.70,120.0,...,曹泽宇,NaN,NaN,NaN,NaN,NaN,NaN,880,28.92,R
1,2,园区运营中心,蛇口网谷,万融大厦,A401-402,办公,简装,1893.60,0.70,135.0,...,曹泽宇,NaN,NaN,NaN,NaN,已签约，9月1日生效,红灯,149,4.90,Y
2,3,园区运营中心,蛇口网谷,万融大厦,BG01/G02/G06/G07/G08/G09/G10/G11/G12,商业,简装,986.06,0.75,240.0,...,商业小组,NaN,NaN,NaN,NaN,NaN,黄灯,361,11.86,R
3,4,园区运营中心,蛇口网谷,万融大厦,A401-402,办公,简装,1893.60,0.70,135.0,...,张可言,NaN,NaN,NaN,NaN,已签约，9月1日生效,NaN,149,4.90,Y
4,5,园区运营中心,蛇口网谷,万融大厦,B座106A,办公,简装,211.90,0.75,135.0,...,曹泽宇,NaN,NaN,NaN,NaN,NaN,红灯,88,2.90,G


In [9]:
vacancy_simplify = vacancy.iloc[:, [2,3,4,7,8,9,11,12,13,15,22,23,24]]
vacancy_simplify.head()

,项目名称,大厦名称,房号,面积,使用率,预计单价,起始日期,空置天数,空置月数,责任人,vacant_days,vacant_month,light
0,蛇口网谷,万融大厦,A座303/305,866.39,0.70,120.0,2019-04-01,876,28.0,曹泽宇,880,28.92,R
1,蛇口网谷,万融大厦,A401-402,1893.60,0.70,135.0,2021-04-01,145,4.0,曹泽宇,149,4.90,Y
2,蛇口网谷,万融大厦,BG01/G02/G06/G07/G08/G09/G10/G11/G12,986.06,0.75,240.0,2020-09-01,357,11.0,商业小组,361,11.86,R
3,蛇口网谷,万融大厦,A401-402,1893.60,0.70,135.0,2021-04-01,124,4.0,张可言,149,4.90,Y
4,蛇口网谷,万融大厦,B座106A,211.90,0.75,135.0,2021-06-01,84,2.0,曹泽宇,88,2.90,G


In [13]:
vacancy_simplify.groupby(['项目名称','大厦名称'])['vacant_days'].nlargest()

项目名称  大厦名称       
W6仓   前海易港中心  87     224
              88     224
              89     224
              90     224
              91     224
                    ... 
高铁网谷  高铁大厦    226    666
              227    666
              228    666
              229    666
              261    300
Name: vacant_days, Length: 222, dtype: int64